In [22]:
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
import pyxpcm
from pyxpcm.models import pcm


def pcm_fit_and_post(time_i=42, K=4, maxvar=0.99):  
    # Define features to use
    # Instantiate the PCM
    
    max_depth = 2000
    z = np.arange(0., -max_depth, -10.)
    features_pcm = {'THETA': z, 'SALT': z}
    features = {'THETA': 'THETA', 'SALT': 'SALT'}
    salt_nc = xr.open_dataset(salt).isel(time=time_i)
    theta_nc = xr.open_dataset(theta).isel(time=time_i)
    big_nc = xr.merge([salt_nc, theta_nc])
    both_nc = big_nc.where(big_nc.coords['Depth'] > 
                           max_depth).drop(['iter', 'Depth', 
                                            'rA', 'drF', 'hFacC'])   
 
    attr_d = {}

    for coord in both_nc.coords:
        attr_d[coord] = both_nc.coords[coord].attrs
        
    lons_new = np.linspace(both_nc.XC.min(), both_nc.XC.max(), 60*4)
    lats_new = np.linspace(both_nc.YC.min(), both_nc.YC.max(), 60)
    # ds = both_nc # .copy(deep=True)
    
    
    ds = both_nc# .interp(coords={'YC': lats_new, 'XC': lons_new}) #, method='cubic')
        
    m = pcm(K=K, features=features_pcm, 
            maxvar=maxvar, separate_pca=False,
            timeit=True, timeit_verb=1)
    # m.fit(ds, features=features, dim='Z') #, inplace=True)
    # m.predict(ds, features=features, dim='Z', inplace=True)
    # m.predict_proba(ds, features=features, dim='Z', inplace=True)
    # m.find_i_metric(ds, inplace=True)
    
    def sanitize():
        #del ds.PCM_LABELS.attrs['_pyXpcm_cleanable']
        #del ds.PCM_POST.attrs['_pyXpcm_cleanable']
        del ds.PCA_VALUES.attrs['_pyXpcm_cleanable']
        

    
    # m.preprocessing_this(ds, dim='Z', feature_name='SALT')
    # X=0
    
    # X, sampling_dims = m.preprocessing(ds, dim='Z', 
    #                                    features={'THETA': 'THETA', 'SALT': 'SALT'})
    
    m.add_pca_to_xarray(ds, dim='Z', 
                        features={'THETA': 'THETA', 'SALT': 'SALT'},
                        action='fit',
                        mask=None, inplace=True)
    
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]

    
    sanitize()
    ds = ds.drop(['SALT', 'THETA'])
    return ds #, #m, X

In [12]:
main_dir = '/Users/simon/bsose_monthly/'
salt = main_dir + 'bsose_i106_2008to2012_monthly_Salt.nc'
theta = main_dir + 'bsose_i106_2008to2012_monthly_Theta.nc'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
ds, m, X = pcm_fit_and_post()

{'THETA': 'THETA', 'SALT': 'SALT'}
  ?.1-preprocess.1-mask: 31 ms
amalgamated option triggered
  ?.1-preprocess.4-xarray: 1 ms
  ?.1-preprocess: 757 ms
{'THETA': 'THETA', 'SALT': 'SALT'}
  fit.1-preprocess.1-mask: 11 ms
amalgamated option triggered
  fit.1-preprocess.4-xarray: 1 ms
  fit.1-preprocess: 407 ms
  fit: 407 ms
  add_pca.xarray: 205 ms


In [23]:
ds = pcm_fit_and_post()

{'THETA': 'THETA', 'SALT': 'SALT'}
  fit.1-preprocess.1-mask: 512 ms
amalgamated option triggered
  fit.1-preprocess.4-xarray: 148 ms
  fit.1-preprocess: 638282 ms
  fit: 638324 ms
  add_pca.xarray: 16957 ms


In [46]:
dim = 'Z'
features= {'THETA': 'THETA', 'SALT': 'SALT'}
features_dict = ds.pyxpcm.feature_dict(m, features=features)
Xlist = []
for feature_in_pcm in features_dict:
    feature_in_ds = features_dict[feature_in_pcm]
    da = ds[feature_in_ds]
    feature_name = feature_in_pcm
    X, z, sampling_dims = m.ravel(da, dim=dim,
                                 feature_name=feature_name)
    X = m._interpoler[feature_name].transform(X, z)
    Xlist.append(X)

In [19]:
Xlist

[<xarray.DataArray (sampling: 10437, Z: 200)>
 dask.array<_interpnd, shape=(10437, 200), dtype=float64, chunksize=(10437, 200), chunktype=numpy.ndarray>
 Coordinates:
     time      datetime64[ns] 2011-08-01T15:12:00
   * sampling  (sampling) MultiIndex
   - YC        (sampling) float64 -76.35 -76.35 -75.53 ... -29.72 -29.72 -29.72
   - XC        (sampling) float64 203.3 204.8 191.3 192.8 ... 356.9 358.4 359.9
   * Z         (Z) float64 0.0 -10.0 -20.0 ... -1.97e+03 -1.98e+03 -1.99e+03,
 <xarray.DataArray (sampling: 10437, Z: 200)>
 dask.array<_interpnd, shape=(10437, 200), dtype=float64, chunksize=(10437, 200), chunktype=numpy.ndarray>
 Coordinates:
     time      datetime64[ns] 2011-08-01T15:12:00
   * sampling  (sampling) MultiIndex
   - YC        (sampling) float64 -76.35 -76.35 -75.53 ... -29.72 -29.72 -29.72
   - XC        (sampling) float64 203.3 204.8 191.3 192.8 ... 356.9 358.4 359.9
   * Z         (Z) float64 0.0 -10.0 -20.0 ... -1.97e+03 -1.98e+03 -1.99e+03]

In [25]:
Xlist[0].values.shape

(10437, 200)

In [24]:
Xlist[1].values.shape

(10437, 200)

In [26]:
X = np.append(Xlist[0].values, Xlist[1].values, axis=1)

In [28]:
X.shape

(10437, 400)

In [ ]:
X, sampling_dims = preprocessing_that(self, ds, dim='Z', features={'THETA': 'THETA', 'SALT': 'SALT'}) #, action='?')


In [45]:
features_dict

{'THETA': 'THETA', 'SALT': 'SALT'}

In [60]:
ds

<xarray.Dataset>
Dimensions:  (XC: 240, YC: 60, Z: 52)
Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -77.16 -76.35 -75.53 ... -31.35 -30.53 -29.72
  * XC       (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Data variables:
    SALT     (Z, YC, XC) float64 nan nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA    (Z, YC, XC) float64 nan nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0

In [61]:
X

<xarray.DataArray (sampling: 10437, n_features: 2)>
array([[-19.58986497,   4.33522835],
       [-19.92733872,   3.97523322],
       [-20.00312861,   4.95466375],
       ...,
       [ 12.38242579,  15.08330564],
       [ 13.00442284,  16.11395374],
       [ 11.50665736,  14.36735465]])
Coordinates:
  * sampling    (sampling) int64 0 1 2 3 4 5 ... 10432 10433 10434 10435 10436
  * n_features  (n_features) int64 0 1

In [71]:
X

<xarray.DataArray (n_samples: 10437, n_features: 5)>
array([[-1.95898650e+01,  4.33522835e+00, -5.51494920e-01,
        -5.12802838e-01, -3.45383332e-01],
       [-1.99273387e+01,  3.97523322e+00, -8.25608172e-01,
        -6.52641690e-01, -6.47994376e-01],
       [-2.00031286e+01,  4.95466375e+00, -7.16851888e-01,
        -3.65836178e-03, -3.67031805e-01],
       ...,
       [ 1.23824258e+01,  1.50833056e+01,  9.85572265e+00,
         7.06589909e+00,  2.94052837e-01],
       [ 1.30044228e+01,  1.61139537e+01,  9.77696349e+00,
         6.99972510e+00,  2.35872499e-01],
       [ 1.15066574e+01,  1.43673546e+01,  1.08186909e+01,
         7.09399007e+00, -1.53175435e-01]])
Coordinates:
  * n_samples   (n_samples) int64 0 1 2 3 4 5 ... 10432 10433 10434 10435 10436
  * n_features  (n_features) <U3 'PC1' 'PC2' 'PC3' 'PC4' 'PC5'

In [76]:
ds

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pca: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pca
Data variables:
    SALT        (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCA_VALUES  (pca, YC, XC) float64 nan nan nan nan ... 0.2941 0.2359 -0.1532

In [5]:
X

'PCA_VALUES'

In [8]:
ds

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pca: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pca
Data variables:
    SALT        (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCA_VALUES  (pca, YC, XC) float64 nan nan nan nan ... 0.2941 0.2359 -0.1532

In [14]:
ds

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pca: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pca
Data variables:
    SALT        (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCA_VALUES  (pca, YC, XC) float64 nan nan nan nan ... 0.2941 0.2359 -0.1532

In [15]:
ds.drop(['SALT', 'THETA'])

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pca: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pca
Data variables:
    PCA_VALUES  (pca, YC, XC) float64 nan nan nan nan ... 0.2941 0.2359 -0.1532

In [16]:
ds

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pca: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pca
Data variables:
    SALT        (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCA_VALUES  (pca, YC, XC) float64 nan nan nan nan ... 0.2941 0.2359 -0.1532

In [17]:
ds

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pca: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pca
Data variables:
    SALT        (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCA_VALUES  (pca, YC, XC) float64 nan nan nan nan ... 0.2941 0.2359 -0.1532

In [20]:
ds

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pca: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pca
Data variables:
    PCA_VALUES  (pca, YC, XC) float64 nan nan nan nan ... 0.2941 0.2359 -0.1532

In [25]:
ds.to_netcdf('trial-full-pc.nc')